# Hands-on with Spark (Structured Streaming)

![spark](https://cdn-images-1.medium.com/max/300/1*c8CtvqKJDVUnMoPGujF5fA.png)

In the previous lesson, we learnt about Spark SQL, Dataframes and Pandas API. In this lesson, we will continue with the Structured Streaming.

Structured Streaming is a scalable and fault-tolerant stream processing engine built on the Spark SQL engine. You can express your streaming computation the same way you would express a batch computation on static data. The Spark SQL engine will take care of running it incrementally and continuously and updating the final result as streaming data continues to arrive.

You can use the Dataset/DataFrame API to express streaming aggregations, event-time windows, stream-to-batch joins, etc. The computation is executed on the same optimized Spark SQL engine. Finally, the system ensures end-to-end exactly-once fault-tolerance guarantees through checkpointing and Write-Ahead Logs. In short, Structured Streaming provides **fast, scalable, fault-tolerant, end-to-end exactly-once** stream processing without the user having to reason about streaming.

Internally, by default, Structured Streaming queries are processed using a micro-batch processing engine, which processes data streams as a series of small batch jobs thereby achieving end-to-end latencies as low as 100 milliseconds and exactly-once fault-tolerance guarantees.

## Installing and Initializing Spark

First, like previously, we'll need to install Spark and its dependencies:

1.   Java 8
2.   Apache Spark with Hadoop
3.   Findspark (used to locate the Spark in the system)


In [8]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.1 pyspark-shell'

# set the options to connect to our Kafka cluster
options = {
    # "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.scram.ScramLoginModule required username="YnJhdmUtZmlzaC0xMTQ2MyQSvwXBuLOQsV1W7YffuC8cDaZcA3fKQwakMhnQGgg" password="MDUxNjc4YzEtYzYxNy00NTE1LWEwNWYtMDBhODRlZmE0OGJm";',
    # "kafka.sasl.mechanism": "SCRAM-SHA-256",
    # "kafka.security.protocol" : "SASL_SSL",
    "kafka.bootstrap.servers": 'localhost:9092',
    "subscribe": 'pizza-orders',
}

In [9]:
# # findspark is only required if you are using a standalone Spark installation (downloaded tar.gz)
# import findspark
# findspark.init()

The below cell may take a few minutes to run. It is slow because:
- Package downloading: Your PYSPARK_SUBMIT_ARGS includes a Kafka package that needs to be downloaded from Maven repositories on first run
- JVM cold start: Initial JVM startup is always slow
- Jupyter overhead: Running in a notebook adds some initialization overhead

In [11]:
# We will only use 2 cores below to speed up creation of the spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LearnSparkStreaming").master("local[2]").getOrCreate()

:: loading settings :: url = jar:file:/Users/dsai/miniconda3/envs/kafka/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/dsai/.ivy2.5.2/cache
The jars for the packages stored in: /Users/dsai/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fe52d6ce-a9c8-4193-a460-926fa782e773;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in cen

In [13]:
spark

## Read and Analyze Kafka stream in "Batch" Mode

Let's start with reading and analyzing our `pizza-orders` kafka topic in the usual "batch" mode of Spark SQL and Dataframes. This is akin to the batch queries we did in the previous lesson. In this case we are taking the messages with the earliest to latest offsets of the topic as a single "batch".

In [40]:
pizza_df = spark.read.format('kafka')\
    .options(**options)\
    .load()

In [41]:
pizza_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [42]:
pizza_df.show()

+--------------------+--------------------+------------+---------+------+--------------------+-------------+
|                 key|               value|       topic|partition|offset|           timestamp|timestampType|
+--------------------+--------------------+------------+---------+------+--------------------+-------------+
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     0|2025-12-09 14:10:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     1|2025-12-09 14:10:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     2|2025-12-09 14:10:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     3|2025-12-09 14:10:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     4|2025-12-09 14:10:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     5|2025-12-09 14:10:...|            0|
|[7B 22 73 68 6F 7.

In [43]:
pizza_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").show()

+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|{"shop": "Circula...|{"id": 0, "shop":...|
|{"shop": "Ill Mak...|{"id": 1, "shop":...|
|{"shop": "Luigis ...|{"id": 2, "shop":...|
|{"shop": "Luigis ...|{"id": 3, "shop":...|
|{"shop": "Luigis ...|{"id": 4, "shop":...|
|{"shop": "Mammami...|{"id": 0, "shop":...|
|{"shop": "Luigis ...|{"id": 1, "shop":...|
|{"shop": "Its-a m...|{"id": 2, "shop":...|
|{"shop": "Marios ...|{"id": 3, "shop":...|
|{"shop": "Mammami...|{"id": 4, "shop":...|
|{"shop": "Circula...|{"id": 5, "shop":...|
|{"shop": "Marios ...|{"id": 6, "shop":...|
|{"shop": "Marios ...|{"id": 7, "shop":...|
|{"shop": "Circula...|{"id": 8, "shop":...|
|{"shop": "Marios ...|{"id": 9, "shop":...|
|{"shop": "Marios ...|{"id": 10, "shop"...|
|{"shop": "Mammami...|{"id": 11, "shop"...|
|{"shop": "Marios ...|{"id": 12, "shop"...|
|{"shop": "Mammami...|{"id": 13, "shop"...|
|{"shop": "Circula...|{"id": 14,

In [44]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, StructType, ArrayType, StructField

In [45]:
pizza_schema = StructType([
  StructField("pizzaName", StringType()),
  StructField("additionalToppings", ArrayType(StringType())),
])

order_schema = StructType([
  StructField("address", StringType()),
  StructField("id", IntegerType()),
  StructField("name", StringType()),
  StructField("phoneNumber", StringType()),
  StructField("shop", StringType()),
  StructField("cost", DoubleType()),
  StructField("pizzas", ArrayType(pizza_schema)),
  StructField("timestamp", LongType()),
])

In [46]:
parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value"))

In [47]:
parsed_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phoneNumber: string (nullable = true)
 |    |-- shop: string (nullable = true)
 |    |-- cost: double (nullable = true)
 |    |-- pizzas: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- pizzaName: string (nullable = true)
 |    |    |    |-- additionalToppings: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- timestamp: long (nullable = true)



In [48]:
parsed_df.show(truncate=False)

+-----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|timestamp              |value                                                                                                                                                                                                                                                                                                                                          |
+-----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

We can use _dot notation_ to select the field within a `Struct`:

In [49]:
parsed_df.select("value.cost").show()

+-----+
| cost|
+-----+
| 9.98|
| 2.61|
| 4.61|
|43.33|
|42.51|
|13.93|
|11.18|
| 4.33|
|40.22|
|37.91|
|25.08|
|34.19|
|22.22|
|24.37|
|29.55|
|23.69|
| 23.3|
| 7.91|
| 0.75|
|  7.6|
+-----+
only showing top 20 rows


Computing the "total revenue" per shop:

In [50]:
parsed_df.groupBy("value.shop").sum("value.cost").show(truncate=False)

+------------------------------------+-----------------------+
|shop                                |sum(value.cost AS cost)|
+------------------------------------+-----------------------+
|Ill Make You a Pizza You Cant Refuse|326.65                 |
|Luigis Pizza                        |369.75                 |
|Mammamia Pizza                      |297.33                 |
|Its-a me! Mario Pizza!              |321.07000000000005     |
|Marios Pizza                        |306.28                 |
|Circular Pi Pizzeria                |239.81                 |
+------------------------------------+-----------------------+



> 1. Count the no. of orders by shop.
> 2. Compute the avg revenue by shop and sort by highest to lowest.

In [51]:
from pyspark.sql.functions import min, max

In [52]:
parsed_df.select(min("timestamp"), max("timestamp")).show(truncate=False)

+-----------------------+-----------------------+
|min(timestamp)         |max(timestamp)         |
+-----------------------+-----------------------+
|2025-12-09 14:10:17.277|2025-12-09 14:37:26.209|
+-----------------------+-----------------------+



## Read and Analyze Kafka stream in "Streaming" Mode

The key idea in Structured Streaming is to treat a live data stream as a table that is being continuously appended. This leads to a new stream processing model that is very similar to a batch processing model. You will express your streaming computation as standard batch-like query as on a static table, and Spark runs it as an incremental query on the *unbounded input table*. Let’s understand this model in more detail.

Consider the input data stream as the “Input Table”. Every data item that is arriving on the stream is like a new row being appended to the Input Table.

![concept](https://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png)

A query on the input will generate the “Result Table”. Every trigger interval (say, every 1 second), new rows get appended to the Input Table, which eventually updates the Result Table. Whenever the result table gets updated, we would want to write the changed result rows to an external sink.

![result table](https://spark.apache.org/docs/latest/img/structured-streaming-model.png)

To illustrate the use of this model, let’s understand the model in context of a word count model. The first lines DataFrame is the input table, and the final wordCounts DataFrame is the result table. Note that the query on streaming lines DataFrame to generate wordCounts is exactly the same as it would be a static DataFrame. However, when this query is started, Spark will continuously check for new data from the socket connection. If there is new data, Spark will run an “incremental” query that combines the previous running counts with the new data to compute updated counts, as shown below.

![example](https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png)




Event-time is the time embedded in the data itself. For many applications, you may want to operate on this event-time.

For example, if you want to get the number of events generated by IoT devices every minute, then you probably want to use the time when the data was generated (that is, event-time in the data), rather than the time Spark receives them.

This event-time is very naturally expressed in this model – each event from the devices is a row in the table, and event-time is a column value in the row. This allows window-based aggregations (e.g. number of events every minute) to be just a special type of grouping and aggregation on the event-time column – each time window is a group and each row can belong to multiple windows/groups. Therefore, such event-time-window-based aggregation queries can be defined consistently on both a static dataset (e.g. from collected device events logs) as well as on a data stream, making the life of the user much easier.

Furthermore, this model naturally handles data that has arrived later than expected based on its event-time. Since Spark is updating the Result Table, it has full control over updating old aggregates when there is late data, as well as cleaning up old aggregates to limit the size of intermediate state data.

In [53]:
pizza_df = spark.readStream.format('kafka')\
    .options(**options)\
    .load()

In [54]:
pizza_df.isStreaming

True

In [55]:
pizza_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [56]:
parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value"))

In [57]:
parsed_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phoneNumber: string (nullable = true)
 |    |-- shop: string (nullable = true)
 |    |-- cost: double (nullable = true)
 |    |-- pizzas: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- pizzaName: string (nullable = true)
 |    |    |    |-- additionalToppings: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- timestamp: long (nullable = true)



In [59]:
query = parsed_df \
    .writeStream \
    .format("console") \
    .start()

# query.awaitTermination() # stops the script from exiting. But this is not required in Jupyter because the kernel stays alive. In real Spark applications, you need awaitTermination() to keep the job running.
# query.isActive
# query.recentProgress
# query.stop()

25/12/09 14:40:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/76/c58h26g508d9m8jvpnjj09dr0000gr/T/temporary-2cde2791-e9bc-492b-ad5b-4728be32ba50. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/12/09 14:40:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+



In [60]:
query.stop()

25/12/09 14:41:03 WARN DAGScheduler: Failed to cancel job group c04356b8-6324-4668-bfb2-c803e2c92f58. Cannot find active jobs for it.
25/12/09 14:41:03 WARN DAGScheduler: Failed to cancel job group c04356b8-6324-4668-bfb2-c803e2c92f58. Cannot find active jobs for it.


In [61]:
query.recentProgress

[{
   "id" : "35bb1dfd-1950-4daa-b808-5f3cf448b389",
   "runId" : "c04356b8-6324-4668-bfb2-c803e2c92f58",
   "name" : null,
   "timestamp" : "2025-12-09T06:40:37.171Z",
   "batchId" : 0,
   "batchDuration" : 754,
   "numInputRows" : 0,
   "inputRowsPerSecond" : 0.0,
   "processedRowsPerSecond" : 0.0,
   "durationMs" : {
     "addBatch" : 33,
     "commitOffsets" : 223,
     "getBatch" : 1,
     "latestOffset" : 330,
     "queryPlanning" : 6,
     "triggerExecution" : 754,
     "walCommit" : 161
   },
   "stateOperators" : [ ],
   "sources" : [ {
     "description" : "KafkaV2[Subscribe[pizza-orders]]",
     "startOffset" : null,
     "endOffset" : {
       "pizza-orders" : {
         "0" : 80
       }
     },
     "latestOffset" : {
       "pizza-orders" : {
         "0" : 80
       }
     },
     "numInputRows" : 0,
     "inputRowsPerSecond" : 0.0,
     "processedRowsPerSecond" : 0.0,
     "metrics" : {
       "avgOffsetsBehindLatest" : "0.0",
       "maxOffsetsBehindLatest" : "0",
   

### In the next few cells, we will provide a complete example of counting the number of orders per pizza shop, as the orders stream in.

In [62]:
# First check if any streams are active
spark.streams.active

In [63]:
# If there are any active streams, stop them
for q in spark.streams.active:
    print(f"Stopping query: {q.name}")
    q.stop()

Stopping query: None


25/12/09 14:41:26 WARN DAGScheduler: Failed to cancel job group 3fbcab69-26cf-42f2-9724-1839c75d222f. Cannot find active jobs for it.
25/12/09 14:41:26 WARN DAGScheduler: Failed to cancel job group 3fbcab69-26cf-42f2-9724-1839c75d222f. Cannot find active jobs for it.


In [64]:
# Complete example with counting
pizza_df = spark.readStream.format('kafka')\
    .options(**options)\
    .load()

parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value")) #.groupBy("value.shop").count()

shop_counts = parsed_df.groupBy("value.shop").count()

# outputMode("complete") rewrites all aggregated results every trigger — good for full snapshots.
# outputMode("update") - only updated rows are written
query = shop_counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

25/12/09 14:41:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/76/c58h26g508d9m8jvpnjj09dr0000gr/T/temporary-269faea1-7544-40a3-9997-b335c2173c63. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/12/09 14:41:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|shop|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+
|                shop|count|
+--------------------+-----+
|Circular Pi Pizzeria|    1|
+--------------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+
|                shop|count|
+--------------------+-----+
|Ill Make You a Pi...|    1|
|        Luigis Pizza|    1|
|Its-a me! Mario P...|    1|
|Circular Pi Pizzeria|    2|
+--------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----+
|                shop|count|
+--------------------+-----+
|Ill Make You a Pi...|    1|
|        Luigis Pizza|    1|
|Its-a me! Mario P...|    2|
|Circular Pi Pizzeria|    2|
+--------------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+-----+
|                shop|count|
+--------------------+-----+
|Ill Make You a Pi...|    1|
|        Luigis Pizza|    1|
|      Mammamia Pizza|    2|
|Its-a me! Mario P...|    3|
|Circular Pi Pizzeria|    3|
+--------------------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/dsai/miniconda3/envs/kafka/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dsai/miniconda3/envs/kafka/lib/python3.11/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dsai/miniconda3/envs/kafka/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [65]:
query.stop()

25/12/09 14:46:08 WARN DAGScheduler: Failed to cancel job group 72e608c8-491d-4dd6-bfbb-f7914023da37. Cannot find active jobs for it.
25/12/09 14:46:08 WARN DAGScheduler: Failed to cancel job group 72e608c8-491d-4dd6-bfbb-f7914023da37. Cannot find active jobs for it.


You have come to the end of this exercise.

To delete the Kafka topic `pizza-orders`, use the command below in your terminal:

`./kafka-topics.sh --delete --topic pizza-orders --bootstrap-server localhost:9092`

To exit Kafka in your terminal, type `exit`.